In [2]:
# ProjectTwoDashboard.ipynb

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import pandas as pd
import urllib.parse

# Import the CRUD module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Data variables
username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
port = 30981
db = "AAC"
collection = "animals"

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password, host, port, db, collection)

# Define the initial query for "Disaster or Individual Tracking"
initial_query = {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}}

# Read data from MongoDB based on the initial query
df = pd.DataFrame.from_records(shelter.read(initial_query))

# Drop the '_id' column for compatibility with Dash DataTable
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add Grazioso Salvare’s logo
image_filename = '/home/felixcarela_snhu/Desktop/Grazioso Salvare Logo.png' # path to the image file in the virtual environment
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([
    html.Div([
        html.Img(
            src=f'data:image/png;base64,{encoded_image}',
            style={'width': '50px', 'height': '50px', 'vertical-align': 'middle', 'margin-right': '10px'}
        ),
        html.B(html.H1('CS-340 Dashboard - Felix Carela', style={'display': 'inline', 'vertical-align': 'middle'}))
    ], style={'text-align': 'center', 'display': 'inline-block', 'width': '100%'}),
    html.Hr(),
    # Add interactive filtering options
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Disaster',  # Set the default value to 'Disaster'
            labelStyle={'display': 'inline-block'}
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=False,
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that the chart and geolocation chart are side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'Reset':
        # Retrieve all records from the MongoDB collection
        dff = pd.DataFrame.from_records(shelter.read({}))
    else:
        query = {}
        if filter_type == 'Water':
            query = {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}}
        elif filter_type == 'Mountain':
            query = {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}}
        elif filter_type == 'Disaster':
            query = {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}}
        dff = pd.DataFrame.from_records(shelter.read(query))
    
    # Drop the '_id' column for compatibility with Dash DataTable
    dff.drop(columns=['_id'], inplace=True)
    return dff.to_dict('records')

# Display the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    
    # Count the occurrences of each breed
    breed_counts = dff['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']
    
    # Select the top 10 results
    top_breeds = breed_counts.nlargest(10, 'count')
    
    return [
        dcc.Graph(
            figure=px.pie(top_breeds, names='breed', values='count', title='Top 10 Results')
        )
    ]

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None:
        return
    dff = pd.DataFrame.from_dict(viewData)
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, dff.columns.get_loc('location_lat')], dff.iloc[row, dff.columns.get_loc('location_long')]], children=[
                dl.Tooltip(dff.iloc[row, dff.columns.get_loc('breed')]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, dff.columns.get_loc('name')])
                ])
            ])
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:25491/
